# Práctica 1 - Naive Bayes

_Pareja 6_
* David Kaack Sánchez
* Carlos César Rodríguez García

## Setup

### Importaciones

In [58]:
from Clasificador import Clasificador, ClasificadorNaiveBayes, ClasificadorNaiveBayesScikit
from Datos import Datos
from EstrategiaParticionado import ValidacionCruzada, ValidacionSimple

import pandas as pd
from sklearn.naive_bayes import CategoricalNB, GaussianNB, MultinomialNB

### Estrategia de particionado

In [59]:
validacion_cruzada = ValidacionCruzada(numeroParticiones=5)
validacion_simple = ValidacionSimple(numeroEjecuciones=5, proporcionTest=30)

### Clasificador generico

In [60]:
clasificador = Clasificador()

### Utilerías

In [61]:
def validacion_naive_bayes_propio(datos, estrategia_particionado, con_laplace):
    return clasificador.validacion(estrategia_particionado, datos, ClasificadorNaiveBayes(con_laplace=con_laplace))

def validacion_naive_bayes(dataset):
    datos = Datos(f"{dataset}.csv")
    
    resultados_vc = validacion_naive_bayes_propio(datos, validacion_cruzada, con_laplace=False)
    resultados_vc_cl = validacion_naive_bayes_propio(datos, validacion_cruzada, con_laplace=True)
    
    resultados_vs = validacion_naive_bayes_propio(datos, validacion_simple, con_laplace=False)
    resultados_vs_cl = validacion_naive_bayes_propio(datos, validacion_simple, con_laplace=True)

    columnas = [
        "Conjunto de datos",
        "Estrategia Particionado",
        "Error Promedio sin CL",
        "Desviación estándar sin CL",
        "Error Promedio con CL",
        "Desviación estándar con CL",
    ]

    filas = [
        (
            dataset,
            "Validación Cruzada",
            resultados_vc[0],
            resultados_vc[1],
            resultados_vc_cl[0],
            resultados_vc_cl[1],
        ),
        (
            dataset,
            "Validación Simple",
            resultados_vs[0],
            resultados_vs[1],
            resultados_vs_cl[0],
            resultados_vs_cl[1],
        )
    ]

    return pd.DataFrame(filas, columns=columnas)

## Apartado 1 - Naive Bayes Propio

### Entrenamiento y clasificación para dataset __heart__

In [62]:
resultados_heart = validacion_naive_bayes("heart")

### Entrenamiento y clasificación para dataset __tic-tac-toe__

In [63]:
resultados_tic_tac_toe = validacion_naive_bayes("tic-tac-toe")

### Analisis de resultados

In [64]:
resultados_naive_bayes_propio = pd.concat([resultados_heart, resultados_tic_tac_toe])
resultados_naive_bayes_propio

,Conjunto de datos,Estrategia Particionado,Error Promedio sin CL,Desviación estándar sin CL,Error Promedio con CL,Desviación estándar con CL
0,heart,Validación Cruzada,0.146110,0.053510,0.146110,0.053510
1,heart,Validación Simple,0.140364,0.019048,0.141091,0.019704
0,tic-tac-toe,Validación Cruzada,0.136267,0.273241,0.139373,0.279523
1,tic-tac-toe,Validación Simple,0.273863,0.035642,0.281704,0.036064


El error promedio en todos los casos es menor al 30%, lo cual consideramos un rendimiento regular. Lo interesante de estos datos es la reducción de la desviación estándar cuando se usa validación simple como estrategia de particionado. En conjuntos de datos pequeños, la validación cruzada puede tener una desviación estándar más alta debido a la limitada cantidad de datos para realizar las divisiones. La validación simple puede ser más estable en tales casos.

#### Efectos de Corrección de Laplace

Podemos ver que no hay mucho impacto de la corrección de Laplace en estos escenarios. Incluso, en la mayoría de escenarios, resulta en un error más grande. Esto puede ocurrir por la ausencia o cantidad reducida de atributos nominales donde se aplique esta corrección.

## Apartado 2 - Naive Bayes Scikit-learn